<a href="https://colab.research.google.com/github/hsym-uec/myWeb_github/blob/main/Simmple_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nintendo Stock data from Stooq.com 7974.JP
# reference https://seanmemo.com/234/

import subprocess
from subprocess import PIPE
from PIL import Image
import tkinter as tk
import tkinter.ttk as ttk
import mplfinance as mpf
import datetime
from datetime import timedelta
from pandas_datareader import data
import pandas as pd
import csvcontrol
import jpholiday
import requests
global tree
global df
global stdata
stdata = pd.read_csv(filepath_or_buffer="data.csv", encoding="shift-jis", sep=",")
global DaysList
DaysList = pd.read_csv(filepath_or_buffer="DaysList.csv", encoding="shift-jis", sep=",")
global date

#土日を祝日に
class SaturdaySundayHoliday(jpholiday.OriginalHoliday):
    def _is_holiday(self, date):
        if date.weekday() >= 5:
            return True

    def _is_holiday_name(self, date):
        return "土日(証券休業日)"

#年末年始を祝日に
class TheYearEndAndNewYearHoliday(jpholiday.OriginalHoliday):
    def _is_holiday(self, date):
        if date.month == 1 and date.day == 1:
            return True
        if date.month == 1 and date.day == 2:
            return True
        if date.month == 1 and date.day == 3:
            return True
        if date.month == 12 and date.day == 31:
            return True
        return False

    def _is_holiday_name(self, date):
        return "年末年始(証券休業日)"
    

def btn_click():
    global df
    global stdata
    global DaysList
    global date

    #入力を受け取る
    code = stdata[stdata["name"] == variable1.get()]["code"].to_string(index=False)
    #code = txt1.get()   
    days = DaysList[DaysList["disp"] == variable2.get()]["num"].to_string(index=False)
    #days = txt2.get()
    #days = input("範囲指定:")
    date = stdata[stdata["name"] == variable1.get()]["date"].to_string(index=False)

    #stooqからデータ取得
    today = datetime.date.today()
    start = today - timedelta(float(days))
    end = today
    df = data.DataReader(code, 'stooq', start, end)

    #グラフ表示
    window = tk.Toplevel()
    window.geometry("800x575")
    window.title(variable1.get() + "    範囲：" + variable2.get())

    fig = 'fig/test.png'
    # イメージ作成
    dfs = df.sort_index()
    mpf.plot(dfs,type='candle',mav=(3, 6, 9),volume=True,savefig=fig)
    img = Image.open(fig)
    h = img.height
    w = img.width
        
    # キャンバス作成
    canvas = tk.Canvas(window, bg="white", height=h, width=w)
    # キャンバス表示
    canvas.place(x=0, y=0)
    # キャンバスにイメージを表示
    img = tk.PhotoImage(file=fig)
    canvas.create_image(0, 0, image=img, anchor=tk.NW)
    
    window.mainloop()


def selected(event):
    for item in tree.selection():
        print(item, tree.item(item))

def change_table():
    global tree
    # Load .csv
    df = pd.read_csv("data.csv", encoding='shift-jis')
    # create Window and Treeview
    table = tk.Tk()
    tree = ttk.Treeview(table, show='headings')
    # set Treeview columns
    tree['column'] = ("No",) + tuple(df)
    # set header
    tree.heading("No", text="No")
    for c in df:
        tree.heading(c, text=c)
    # set cells on the row
    for i, row in enumerate(df.itertuples()):
        tree.insert("", "end", tags=i, values=row)
    # set layout
    tree.pack()
    # bind action
    tree.bind('<<TreeviewSelect>>', selected)
    btn3 = tk.Button(table,text='グラフ表示')
    btn3.pack(side=tk.BOTTOM, fill=tk.X)
    # set loop
    root.mainloop()
    
def change_table2():
    proc = subprocess.run(['python', 'csvcontrol.py'], stdout=PIPE, stderr=PIPE)
    #csvcontrol()


def load():
    global stdata
    stdata = pd.read_csv(filepath_or_buffer="data.csv", encoding="shift-jis", sep=",")
    global opt1
    opt1.destroy()
    opt1 = tk.OptionMenu(root, variable1, *stdata["name"])
    opt1.config(width=20)
    #opt1.pack()
    opt1.place(x=80, y=70)

def show_benefit():
    #画面作成
    global stdata
    sum = 0
    frame_sb = tk.Tk()
    frame_sb.title("損益")
    frame_sb.geometry("450x450")
    tree = ttk.Treeview(frame_sb)
    tree["columns"] = (1, 2, 3, 4)
    tree["show"] = "headings"

    tree.column(1, width=100)
    tree.column(2, width=100)
    tree.column(3, width=100)
    tree.column(4, width=100)

    tree.heading(1, text="銘柄")
    tree.heading(2, text="購入日")
    tree.heading(3, text="損益")
    tree.heading(4, text="合計")

    #表の中身生成とエラー処理
    for index, row in stdata.iterrows():
        code = row[1]
        today = datetime.date.today()
        start = row[3]
        end = today
        if row[3] == today.strftime('%Y-%m-%d'):
            tree.insert("", "end", values=(row["name"], row[3], "確定前", sum))
        elif jpholiday.is_holiday(datetime.datetime.strptime(row[3], '%Y-%m-%d')):
            tree.insert("", "end", values=(row["name"], row[3], "非営業日", sum))
        else:
            df1 = data.DataReader(code, 'stooq', start, end)
            benefit = (df1["Close"][0] - df1["Close"][row[3]])*row[2]
            sum += benefit
            tree.insert("", "end", values=(row["name"], row[3], benefit, sum))

    vbar = ttk.Scrollbar(frame_sb, orient = 'v', command = tree.yview)

    tree.configure(yscrollcommand = vbar.set) 

    tree.pack(side = tk.LEFT)
    tree.place(x = 15, y = 15, height = 200) 

    vbar.pack(side = tk.LEFT, fill = tk.Y) 
    vbar.place(x = 420, y = 15, height = 200) 

    frame_sb.mainloop()

def show_all():
    global tree
    url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    r = requests.get(url)
    with open('data_j.xls', 'wb') as output:
        output.write(r.content)

    stocklist = pd.read_excel("./data_j.xls")
    stocklist.loc[stocklist["市場・商品区分"]=="市場第一部（内国株）",
                ["コード","銘柄名","33業種コード","33業種区分","規模コード","規模区分"]
                ]
    
    # Load .csv
    df = pd.read_excel("data_j.xlsx")
    # create Window and Treeview
    table = tk.Tk()
    tree = ttk.Treeview(table, show='headings')
    # set Treeview columns
    tree['column'] = ("No",) + tuple(df)
    # set header
    tree.heading("No", text="No")
    for c in df:
        tree.heading(c, text=c)
    # set cells on the row
    for i, row in enumerate(df.itertuples()):
        tree.insert("", "end", tags=i, values=row)
    # set layout
    tree.pack()
    # bind action
    tree.bind('<<TreeviewSelect>>', selected)
    # set loop
    root.mainloop()

#ウィンドウ表示
root = tk.Tk()
root.title("株価表示")
root.geometry("300x200")

#テキスト
#lbl1 = tk.Label(text='銘柄追加')
#lbl1.place(x=30, y=40)
lbl2 = tk.Label(text='銘柄：')
lbl2.place(x=30, y=70)
lbl3 = tk.Label(text='範囲：')
lbl3.place(x=30, y=100)

#テキストボックス
#txt1 = tk.Entry(width=20)
#txt1.place(x=80, y=40)
#txt2 = tk.Entry(width=20)
#txt2.place(x=80, y=100)

#プルダウン
variable1 = tk.StringVar(root)
global opt1
opt1 = tk.OptionMenu(root, variable1, *stdata["name"])
opt1.config(width=20)
#opt1.pack()
opt1.place(x=80, y=70)

variable2 = tk.StringVar(root)
opt2 = tk.OptionMenu(root, variable2, *DaysList["disp"])
opt2.config(width=20)
#opt2.pack()
opt2.place(x=80, y=100)

#ボタン
btn = tk.Button(text='グラフ表示', command=btn_click)
btn.pack(side=tk.BOTTOM, fill=tk.X)
btn2 = tk.Button(text='銘柄確認', command=change_table2)
btn2.place(x=30,y=30)
btn3 = tk.Button(text='更新', command=load)
btn3.place(x=120,y=30)
btn4 = tk.Button(text='損益表示', command=show_benefit)
btn4.pack(side=tk.BOTTOM, fill=tk.X)
btn5 = tk.Button(text='銘柄一覧', command=show_all)
btn5.pack(side=tk.BOTTOM, fill=tk.X)


root.mainloop()

In [ ]:
df

In [ ]:
df["Close"].plot(figsize=(20,10), color="red")

In [ ]:
# 前日と前々日の差額
df["Close"][0] - df["Close"][date]

In [ ]:
a = '2023-05-16'
print(date)
df["Close"][date]

In [ ]:
dfc = df.sort_index()
mpf.plot(dfc,type='candle',mav=(3, 6, 9),volume=True)